# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [2]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.5 MB/s eta 0:00:00


In [3]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [4]:

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [11]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="search", #give it a meaningful name,
    embedding=embeddings,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········
OpenAI API key: ··········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [5]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

In [19]:
# Initialize counter
null_or_empty_count = 0

# Iterate through records
for record in data:
    if 'updated_at' in record and (not record['updated_at'] or record['updated_at'].strip() == ''):
        null_or_empty_count += 1

print(f"Number of records with null or empty 'updated_at': {null_or_empty_count}")


Number of records with null or empty 'updated_at': 0


In [7]:
from collections import defaultdict

# Dictionary to store counts of each data type
data_type_counts = defaultdict(int)

# Iterate through records
for record in data:
    if 'updated_at' in record:
        data_type = type(record['updated_at']).__name__  # Get data type name (e.g., 'str', 'int', 'float', 'NoneType')
        data_type_counts[data_type] += 1

# Print the counts of each data type
for data_type, count in data_type_counts.items():
    print(f"Data type '{data_type}': {count} occurrences")

Data type 'str': 5 occurrences


### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [12]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    metadata["name"] = record.get("name")
    metadata["summary"] = record.get("summary")
    metadata["url"] = record.get("url")
    metadata["category"] = record.get("category")
    metadata["updated_at"] = record.get("updated_at")

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [13]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="search",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [15]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

# Configure logging to show INFO level logs from the 'langchain.retrievers.multi_query' module
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Define a chat prompt template for contextual information retrieval tasks
LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

# Define a prompt template for documents, specifying the source name and page content
LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

# Function to combine multiple documents into a single formatted string separated by document_separator
def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    # Format each document using format_document function with specified document_prompt
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    # Join formatted document strings using document_separator
    return document_separator.join(doc_strings)

# RunnableParallel instance that combines context retrieval with document formatting
_context = RunnableParallel(
    context=retriever | _combine_documents,  # Context includes document retrieval and formatting
    question=RunnablePassthrough(),  # Passes questions through unchanged
)

# Define a processing chain combining context retrieval, prompting, and language model inference
chain = _context | LLM_CONTEXT_PROMPT | llm

# Invoke the processing chain with a specific question
ans = chain.invoke("what is the nasa sales team?")

# Print the resulting answer from the language model inference
print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling the company's products and services to potential clients in the North America and South America regions. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company consistently delivers high-quality products and services to its clients.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [16]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    metadata["name"] = record.get("name")
    metadata["summary"] = record.get("summary")
    metadata["url"] = record.get("url")
    metadata["category"] = record.get("category")
    metadata["updated_at"] = record.get("updated_at")

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=400, chunk_overlap=200 #reduced the chunk size and overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

In [17]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="search",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=1, openai_api_key=OPENAI_API_KEY) #changed temperature to 1

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

In [21]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

# Configure logging to show INFO level logs from the 'langchain.retrievers.multi_query' module
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Define a chat prompt template for contextual information retrieval tasks
LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say "if you don't know, how would I know?". Be straight to the point and throw a joke at the end.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

# Define a prompt template for documents, specifying the source name and page content
LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

# Function to combine multiple documents into a single formatted string separated by document_separator
def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    # Format each document using format_document function with specified document_prompt
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    # Join formatted document strings using document_separator
    return document_separator.join(doc_strings)

# RunnableParallel instance that combines context retrieval with document formatting
_context = RunnableParallel(
    context=retriever | _combine_documents,  # Context includes document retrieval and formatting
    question=RunnablePassthrough(),  # Passes questions through unchanged
)

# Define a processing chain combining context retrieval, prompting, and language model inference
chain = _context | LLM_CONTEXT_PROMPT | llm

# Invoke the processing chain with a specific question
ans = chain.invoke("what is the nasa sales team?")

# Print the resulting answer from the language model inference
print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the roles and responsibilities of the NASA sales team?', '2. How can I learn more about the structure and function of the NASA sales team?', '3. Is there a document or resource that outlines the goals and objectives of the NASA sales team?']


---- Answer ----
The NASA sales team is responsible for the North America and South America region within the Americas, including the United States, Canada, Mexico, Central and South America. It is led by two Area Vice-Presidents, Laura Martinez and Gary Johnson.


In [19]:
# Invoke the processing chain with a specific question
ans = chain.invoke("what is nasa?")

# Print the resulting answer from the language model inference
print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on nasa?', '2. What data do you have on nasa?', '3. Could you give me details on nasa?']


---- Answer ----

NASA (North America South America) is an Area Vice-Presidency within the Americas region of the sales organization. It is responsible for overseeing sales in both North America and South America, with Laura Martinez as the Area Vice-President for North America and Gary Johnson as the Area Vice-President for South America.


This output looked strange to me so I asked more detailed information below:

In [22]:
# Invoke the processing chain with a specific question
ans = chain.invoke("who owns Mars?")

# Print the resulting answer from the language model inference
print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Who is the owner of Mars?', '2. Can you tell me the name of the person who owns Mars?', '3. What is the identity of the person holding ownership over Mars?']


---- Answer ----
if you don't know, how would I know?
    Just kidding, no one currently owns Mars, but maybe one day we can apply these Intellectual Property policies to space travel and colonization.


It's